In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/CPDP_GroupProject

/content/drive/MyDrive/CPDP_GroupProject


In [ ]:
ls

' FeatureVectors.ipynb'   ImageSearch.ipynb              __pycache__/
 FeatureVectors.py       ' ImageSearch_Parallel.ipynb'   QuerySearch.ipynb
 Image_Database/          ImageSearch_parallel.py        QuerySearch.py
 Image_Filters/           ImageSearch_Serial.ipynb
 Image_Histograms/        ImageSearch_Serial.py


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Image-Search-using-Parallel-Computing-master')
sys.path.append('/content/drive/MyDrive/CPDP_GroupProject/Image_Histograms/')
sys.path.append('/content/drive/MyDrive/CPDP_GroupProject/ConvolutionalFilters /')
sys.path.append('/content/drive/MyDrive/CPDP_GroupProject/FeatureVectors.py/')
sys.path.append('/content/drive/MyDrive/CPDP_GroupProject/QuerySearch.py')

In [ ]:
import os
import sys
import threading
import concurrent.futures
import time
import cv2
import matplotlib.pyplot as plt




In [ ]:
from FeatureVectors import FeatureVectors
from QuerySearch import QuerySearch


In [ ]:
def extractFeatureVectors(image_path):
    # Extracts feature vectors for input image
    
    with cv2.imread(image_path) as image:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (500, 500))
        featureVectors = FeatureVectors(image)
        vectors = featureVectors.getFeatureVector()

        imageName = image_path.split("/")[-1]
        return [imageName, vectors]


In [ ]:
import threading

def ThreadedFeatureExtraction(images_list):
    # Performing feature extraction of database images using multithreading

    features = {}
    threads = []

    for image_path in images_list:
        t = threading.Thread(target=extractFeatureVectors, args=(image_path,))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()
        imageName, vectors = t.result()
        features[imageName] = vectors

    return features




In [ ]:
def getImg(img):
    image_db_path = os.path.join("Image_Database", img)
    with open(image_db_path, 'rb') as f:
        image_bytes = f.read()
    image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)




In [ ]:
def ImageSearch(queryImage):
    # Performs Image Search using Query image

    image_db_path = "Image_Database/"
    image_paths = [image_db_path + img for img in os.listdir(image_db_path)]

    features = ThreadedFeatureExtraction(image_paths)

    queryImage_path = image_db_path + queryImage
    imageName, queryVector = extractFeatureVectors(queryImage_path)

    search = QuerySearch(queryVector, features)

    results = search.performSearch()
    results.sort(key=lambda res: res[1])

    return results
